In [2]:
!pip install transformers onnx onnxruntime


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 18.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.1 MB/s eta 0:00:0000:0100:01
  error: subprocess-exited-with-error
  
  × Building wheel for safetensors (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [25 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-c

In [3]:
!python -m transformers.onnx --model=bert-base-uncased model

/Users/juniverse/opt/anaconda3/envs/vision00/bin/python: Error while finding module specification for 'transformers.onnx' (ModuleNotFoundError: No module named 'transformers')


기본적으로 weight 를 저장하기위해 fp32 ( floating points 32-bits)활용.

fp32 대신에 fp16, int8과 같은 타입을 활용해서 손실이 생기지만 속도향상.
학습에서도 fp16만 사용하거나 섞어서 사용하는 방식있음.

## ONNX quantization

ONNX 에서 제공하는 quantization은 & 8 bit quantization 지원함.

https://onnxruntime.ai/docs/performance/model-optimizations/quantization.html



In [4]:
import onnx
from onnxruntime.quantization import quantize_dynamic

model_fp32 = 'model/model.onnx'
model_quant = 'model/model.quantized.onnx'
quantize_dynamic(model_fp32, model_quant)

FileNotFoundError: [Errno 2] No such file or directory: 'model/model.onnx'

In [4]:
!ls -alh model

total 523M
drwxr-xr-x 2 root root 4.0K Jul 23 02:51 .
drwxr-xr-x 1 root root 4.0K Jul 23 02:50 ..
-rw-r--r-- 1 root root 418M Jul 23 03:09 model.onnx
-rw-r--r-- 1 root root 105M Jul 23 03:09 model.quantized.onnx


In [5]:
import time
import onnxruntime as ort
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
encoded_input = tokenizer(" ".join(["hello"]*510), return_tensors='np')

session = ort.InferenceSession("model/model.onnx", providers=['CPUExecutionProvider'])
quantized_session = ort.InferenceSession("model/model.quantized.onnx", providers=['CPUExecutionProvider'])

def benchmark(sess):
  start = time.time()
  for _ in range(10):
    output = sess.run(None, input_feed=dict(encoded_input))
  end = time.time()
  print(end - start)

benchmark(session)
benchmark(quantized_session)

17.39389729499817
12.877118349075317
